In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import  LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,r2_score,classification_report
import yfinance as yf
import pandas_ta as  ta
import numpy as np
from scipy.signal import argrelextrema


In [2]:
# data = yf.download("EURUSD=X",start='2023-01-01',end='2024-07-01', interval='1h' )

data = pd.read_csv('EURUSD.csv')
# df = pd.read_csv('D_RELIANCE1.csv')
# data.head() # data.describe()
df = data.drop('Volume',axis=1)
# df.ta.indicators()

# df = pd.DataFrame(df)
# df['Date'] = pd.to_datetime(df['Datetime'],utc=True)
# df.set_index('Date',inplace=True)
#df["Date"] = int(df['Date'])

# df['Date'] = df.index.astype('int64') // 10**9  # Convert to seconds

df['ATR'] = df.ta.atr(length=20)
df['RSI'] =  df.ta.rsi()
df['AVG'] =  df.ta.midprice(length = 1)
df['20SMA'] = df.ta.sma(length=20)
df['40SMA'] = df.ta.sma(length=40)
df['100SMA'] = df.ta.sma(length=100)
df['160SMA'] = df.ta.sma(length=160)

from scipy.stats import linregress
def get_slope(arr):
    x = pd.array(arr)
    y = np.arange(len(x))
    slopes, x,y,z,w = linregress(x,y)
    return slopes

backrollingN = 10
# #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df['slope20SMA'] = df['20SMA'].rolling(window=backrollingN).apply(get_slope,raw=True)
df['slope40SMA'] = df['40SMA'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeSMA100'] = df['100SMA'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeSMA100'] = df['160SMA'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['AverageSlope'] = df['AVG'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['RSISlope'] = df['RSI'].rolling(window=backrollingN).apply(get_slope, raw=True)


df = df.dropna()

In [3]:
max_idx = argrelextrema(data['Close'].values, np.greater, order=5)[0]
min_idx = argrelextrema(data['Close'].values, np.less, order=5)[0]

max_idx =max_idx[max_idx<df.shape[0]]
min_idx =min_idx[min_idx<df.shape[0]]


labels = np.zeros(len(df))  # Initialize with non-pivot labels
labels[max_idx] = 1  # Set labels for pivot highs
# labels[min_idx] = -1  # Set labels for pivot lows
df['labels']= labels

C:\Users\Admin\AppData\Local\Temp\ipykernel_11612\1525720504.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels']= labels


In [4]:

# df = df.drop("Datetime",axis=1)
# df = df.drop('lable',axis=1)

x_lin = df.drop("Close",axis=1)
y_lin = df['Close']

x_log = df.drop("labels",axis=1)
y_log = df['labels']

# df



In [5]:
# df['labels']

In [6]:
X_train, x_test,Y_train,y_test =  train_test_split(x_lin,y_lin, test_size=0.2, random_state = 42)

In [7]:
lin = LinearRegression()
lin.fit(X_train,Y_train)

ValueError: could not convert string to float: '2023-07-18 23:00:00+01:00'

In [ ]:
y_lin_pred = lin.predict(x_test)
lin.score(x_test,y_test)
r2_score(y_test,y_lin_pred)
# confusion_matrix(y_lin_pred,y_test)
# classification_report(y_test,y_lin_pred)

In [ ]:
X_train, x_test,Y_train,y_test =  train_test_split(x_log,y_log, test_size=0.2, random_state = 42)

In [ ]:
log = LogisticRegression(multi_class= 'multinomial',max_iter=100)
log.fit(X_train,Y_train)
log.score(x_test,y_test)

In [ ]:
y_pred = log.predict(x_test)
# y_test.drop('Date')
score = log.score(x_test,y_test)
print(score)
# print(y_test)
# print(y_pred)
y_test.count()

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)
print(f"r2:{r2}\nmse:{mse}")

In [ ]:
cm =confusion_matrix(y_pred,y_test)
plt.figure(figsize=(4,4))
sns.heatmap(cm,annot=True,cmap='Blues_r',fmt='.1f',linewidth=5,square=True)
# sns.heatmap(cm ,annot=True,cmap='Blues_r', fmt ='.3f',linewidth=5,square= True)


# 

In [ ]:
print(y_pred[:300])

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
classification_report(y_test,y_pred)